# Libraries, Modules, and Configuration File

In [ ]:
import pandas as pd
import sys
sys.path.append('/root/ml_process_feb23/')
import src.util as utils
config = utils.load_config()
from sklearn.model_selection import train_test_split

: 

# Data Collection

In [ ]:
# Loading dataset
credit_data = pd.read_csv(config["dataset_original_path"])
credit_data


: 

# Data Definition


In [ ]:
# Check the DataFrame information
credit_data.info(verbose = False)


: 

From the information shown above, I comprehend a general 
insight into the `credit_data`.

It has 1319 rows and 12 features, which consist of 9 numerical features, and 3 numerical categorical features.








In [ ]:
# Check the features name
credit_data.columns

: 

Features definition:


*   `card`: Acceptance status of credit card application
*   `reports`: Number of major derogatory reports
*   `age`: Age, in years plus twelfths of a year
*   `income`: Yearly income (divided by 10,000)
*   `share`: Ratio of monthly credit card expenditure to yearly income
*   `expenditure`: Average monthly credit card expenditure
*   `owner`: Home ownership
*   `selfempl`: Self-employed status, "no" means an employee to a company
*   `dependents`: Number of dependents
*   `months`: Months living at current address
*   `majorcards`: Number of major credit cards held
*   `active`: Number of active credit accounts








# Data Validation


## Checking Null Values and Data Types in Each Feature

In [ ]:
# Check the detailed DataFrame information
credit_data.info()

: 

There is no null value in this dataset.

The object (string) data types will be later converted to numeric.

## Checking The General Data Distribution

In [ ]:
# Check the distribution of numerical features
credit_data.describe()


: 

Notes on the numerical data.
*  The counts in the entire features are the same value, meaning that `credit_data` has no null value.
* The minimum age recorded is 2 months old, which is consciously impossible to apply for a credit card at that age.
* Applicant(s) who have zero expenditure will also be checked thoroughly later to make sure the data is correctly filled in.






In [ ]:
# Check the distribution of categorical features
credit_data.describe(include=['O'])

: 

All the categorical features have binary value, "yes" and "no".

## Data Deletion

The legal age to have a credit card is 18 years old, thus the data in feature `age` valued at less than 18 will be dropped.

I decide to do this deletion work in advance to limit the minimum data range in the data defense.

In [ ]:
def del_rows(dataset, col, value):
    """A function for deleting data in numeric column,
    specify by the value"""
    
    dataset = dataset.loc[dataset[col] >= value].reset_index(drop = True)
    return dataset

: 

In [ ]:
# Execute the data deletion function
credit_data = del_rows(credit_data, 'age', 18)

: 

# Data Defense

Create a defense mechanism for the input data.

In [ ]:
def check_data(input_data, config):

    # Check data types
    assert input_data.select_dtypes("object").columns.to_list() == config["object_columns"], "input error, please fill the column(s) with 'yes' or 'no'"
    assert input_data.select_dtypes("int").columns.to_list() == config["int_columns"], "input error, please fill the column(s) with any numeric character."
    assert input_data.select_dtypes("float").columns.to_list() == config["float_columns"], "input error, please fill the column(s) with any numeric (decimal value is allowed) character."

    # Check range of data
    assert set(input_data[config["object_columns"][1]]).issubset(set(config["range_owner"])), "an error occurs in owner range."
    assert set(input_data[config["object_columns"][2]]).issubset(set(config["range_selfemp"])), "an error occurs in selfemp range."
    assert input_data[config["int_columns"][0]].between(config["range_reports"][0], config["range_reports"][1]).sum() == len(input_data), "an error occurs in reports range."
    assert input_data[config["float_columns"][0]].between(config["range_age"][0], config["range_age"][1]).sum() == len(input_data), "an error occurs in age range."
    assert input_data[config["float_columns"][1]].between(config["range_income"][0], config["range_income"][1]).sum() == len(input_data), "an error occurs in income range."
    assert input_data[config["float_columns"][2]].between(config["range_share"][0], config["range_share"][1]).sum() == len(input_data), "an error occurs in share range."
    assert input_data[config["float_columns"][3]].between(config["range_expenditure"][0], config["range_expenditure"][1]).sum() == len(input_data), "an error occurs in expenditure range."
    assert input_data[config["int_columns"][1]].between(config["range_dependents"][0], config["range_dependents"][1]).sum() == len(input_data), "an error occurs in dependents range."    
    assert input_data[config["int_columns"][2]].between(config["range_months"][0], config["range_months"][1]).sum() == len(input_data), "an error occurs in months range."
    assert input_data[config["int_columns"][3]].between(config["range_majorcards"][0], config["range_majorcards"][1]).sum() == len(input_data), "an error occurs in majorcards range."
    assert input_data[config["int_columns"][4]].between(config["range_active"][0], config["range_active"][1]).sum() == len(input_data), "an error occurs in active range."
   


: 

In [ ]:
# checking the data defense function
check_data(credit_data, config)

: 

# Data Splitting

In [ ]:
def split_input_output(dataset,
                       target_column,
                       save_file = True,
                       return_file = True):
    """Divide the data into its dependent variable/target (y-axis) and independent/predictor (x-axis) ones,
    input_df = predictors while output_df = target"""

    output_df = dataset[target_column]
    input_df = dataset.drop([target_column],
                            axis = 1)

    if save_file:  
        utils.pkl_dump(output_df, config["dataset_output_df_path"])
        utils.pkl_dump(input_df, config["dataset_input_df_path"])
    
    if return_file:
        return output_df, input_df


def split_train_test(x, y, TEST_SIZE):
    """Split the data into the training and test data,
    stratify parameter is activated,
    this function will be reproduced later as the data-splitting process further"""
    
    x_train, x_test, y_train, y_test = train_test_split(x,
                                                        y,
                                                        test_size=TEST_SIZE,
                                                        random_state=46,
                                                        stratify=y)

    return x_train, x_test, y_train, y_test

def split_data(data_input, 
               data_output, 
               save_file = True,
               return_file=True, 
               TEST_SIZE=0.17):
    """Split the data into the training, validation and test data,
    first split process will return into the train and test data,
    the train data resulted from the first will be splitted further into the train and validation data.
    The TEST_SIZE 0.17 resulting the test data 17% proportion from the dataset length,
    while the proportion of train data and validation data are respectively 69% and 14%. 
    All files returned from this function are saved into pickle files."""

    x_train, x_test, y_train, y_test = split_train_test(data_input, 
                                                        data_output,
                                                        TEST_SIZE)

    x_train, x_valid, y_train, y_valid = split_train_test(x_train,
                                                          y_train,
                                                          TEST_SIZE)
    
    if save_file:
        utils.pkl_dump(x_train, config["dataset_train_path"][0])
        utils.pkl_dump(y_train, config["dataset_train_path"][1])
        utils.pkl_dump(x_valid, config["dataset_valid_path"][0])
        utils.pkl_dump(y_valid, config["dataset_valid_path"][1])
        utils.pkl_dump(x_test, config["dataset_test_path"][0])
        utils.pkl_dump(y_test, config["dataset_test_path"][1])

    if return_file:
        return x_train, y_train, \
            x_valid, y_valid, \
            x_test, y_test


: 

In [ ]:
# Execute the data-splitting functions

output_df, input_df = split_input_output(credit_data,
                                        target_column = "card", 
                                        save_file = False)

x_train, y_train, x_valid, y_valid, x_test, y_test = split_data(input_df,
                                                               output_df)


: 

# Data Review

The "credit" feature in `credit_data` is set as the dependent variable (saved as output_df), while the rest features are the independent variables (saved as input_df).


In [ ]:
# Recheck the division of the dependent and independent variables from the `credit_data`
print(f'The dependent variable (target):\n{output_df} \n')
print("-------------------------------------\n")
print(f'The independent variables (predictors):\n {input_df}')

: 

Both the `input_df` and `output_df` are split into three data that will be used for training, validation, and testing the model.

The training data has the majority proportion.




In [ ]:
# Inspect the data-splitting result
print(f"input_df shape: {input_df.shape}")
print(f"x_train shape: {x_train.shape}")
print(f"x_valid shape:{x_valid.shape}")
print(f"x_test shape: {x_test.shape}\n")
print("-------------------------------------\n")
print(f"output_df shape: {output_df.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_valid shape: {y_valid.shape}")
print(f"y_test shape: {y_test.shape}")


: 

The stratified splitting method applies to the `output_df` to keep the value proportion approximately the same.

In [ ]:
# Recheck the data stratification
print(f"y_train value proportion\n{y_train.value_counts(normalize = True)}")
print("-------------------------------------\n")
print(f"y_valid value proportion\n{y_valid.value_counts(normalize = True)}")
print("-------------------------------------\n")
print(f"y_test value proportion\n{y_test.value_counts(normalize = True)}")

: 